# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sns
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = '../output_data/cities.csv'
city_file  = output_data_file
city_df = pd.read_csv(city_file)
city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Jamestown,1,US,1586654195,48,42.10,-79.24,42.01,3.36
1,1,Kalianget,44,ID,1586654371,76,-7.05,113.93,80.87,3.29
2,2,Uvat,1,RU,1586654372,90,59.14,68.90,33.80,4.63
3,3,Atuona,38,PF,1586654271,64,-9.80,-139.03,82.67,5.21
4,4,Busselton,92,AU,1586654244,59,-33.65,115.33,73.99,9.22
...,...,...,...,...,...,...,...,...,...,...
564,564,Natchez,1,US,1586654864,64,31.56,-91.40,69.80,5.82
565,565,Vestmannaeyjar,100,IS,1586654903,84,63.44,-20.27,43.20,8.34
566,566,Tocopilla,39,CL,1586655164,50,-22.09,-70.20,67.39,3.47
567,567,Puro,1,PH,1586655165,58,12.48,123.38,85.48,4.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmap
gmaps.configure(api_key=g_key)

In [4]:
#Store latitude and longitude in locations, set humidity as weight
locations = city_df[['Lat', 'Lng']]
weight = city_df['Humidity']

fig = gmaps.figure()
#make a layer
humidity = gmaps.heatmap_layer(locations, weights=weight)
#add the layer
fig.add_layer(humidity)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
purfect_weather = city_df.loc[(city_df["Cloudiness"] == 0) & \
                              (city_df["Humidity"]>= 50) & \
                              (city_df["Cloudiness"]<= 60) & \
                              (city_df["Max Temp"]>= 70) & (city_df["Max Temp"]<= 90)]
purfect_weather_df = pd.DataFrame(purfect_weather).copy()
reduced_df = purfect_weather_df   


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = reduced_df
hotel_df['Hotel Name'] = ''
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
336,336,Nouakchott,0,MR,1586654826,53,18.09,-15.98,74.95,17.72,
351,351,Cockburn Town,0,TC,1586654845,77,21.46,-71.14,79.61,13.73,
409,409,Ambilobe,0,MG,1586654932,84,-13.20,49.05,72.12,3.96,
460,460,Mackay,0,AU,1586655002,62,-21.15,149.20,88.00,9.17,
473,473,Thinadhoo,0,MV,1586655024,64,0.53,72.93,84.18,5.50,
511,511,Celestún,0,MX,1586655085,80,20.87,-90.40,80.17,14.14,


In [7]:
#create a params dict that will be updated with new hotel each row iteration
target_type = "hotel"

params = {
    "radius": 50000,
    "types": target_type,
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing incorporate a try-except to skip any missing  data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [8]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
336,336,Nouakchott,0,MR,1586654826,53,18.09,-15.98,74.95,17.72,AZALAÏ HOTEL NOUAKCHOTT (MARHABA )
351,351,Cockburn Town,0,TC,1586654845,77,21.46,-71.14,79.61,13.73,Bohio Dive Resort
409,409,Ambilobe,0,MG,1586654932,84,-13.20,49.05,72.12,3.96,Hotel National
460,460,Mackay,0,AU,1586655002,62,-21.15,149.20,88.00,9.17,Tandara Hotel Motel
473,473,Thinadhoo,0,MV,1586655024,64,0.53,72.93,84.18,5.50,Ayada Maldives
511,511,Celestún,0,MX,1586655085,80,20.87,-90.40,80.17,14.14,Hotel Xixim


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
locations = hotel_df[['Lat', 'Lng']]
fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))